In [116]:
import pandas as pd
from pyspark.sql import SparkSession

In [111]:
big_list = []
with open('YelpData.txt', 'rb') as f:
    data = f.readlines()

for partial_data in data:
    df = pd.read_json(partial_data)
    for column in df.columns:
        temp = list(df[column])
        temp.insert(0,column)
        big_list.append(temp)
new_df = pd.DataFrame(big_list,columns=["key", "website", "restaurant","date", "review", "rating", "num_votes"])

In [124]:
spark = SparkSession.builder.appName('restaurant_reviews').getOrCreate()
spark_df = spark.createDataFrame(new_df)
spark_df.write.saveAsTable("yelp")

In [132]:
spark_df.groupBy("restaurant").count().show()

+--------------------+-----+
|          restaurant|count|
+--------------------+-----+
|        Gorgers Subs|  140|
|            Tamarind|  106|
|     Napoli Pizzeria|   48|
|Luna Inspired Str...|   90|
|Applebee’s Grill ...|   27|
|  Buffalo Wild Wings|   31|
|              Subway|    1|
|       Stella’s Barn|   27|
|Capital State Kit...|   20|
|         Sushi Osaka|    9|
|     Gangnam Station|   31|
|Spring Buffet & G...|   37|
| Dapper’s Restaurant|    6|
|Taste of Thai Exp...|  166|
|           Taco Bell|   16|
|         Pizza Aroma|   55|
|       Taste of Thai|  180|
|         Atrium Cafe|    3|
|Little Caesars Pizza|    3|
|         Nikki Green|   33|
+--------------------+-----+
only showing top 20 rows

